[Mountain View Spatiotemporal Analysis Working Group](http://www.meetup.com/Mountain-View-Spatiotemporal-Analysis-Working-Group/)

# Load dataset

In [1]:
import os
import pandas

source_folder = os.path.expanduser('~/Datasets/USA/NYC')
source_path = os.path.join(source_folder, 'NYC-311ServiceRequests-20140203.h5')
nyc_service_requests = pandas.read_hdf(source_path, 'raw')
nyc_service_request_locations = nyc_service_requests[
    ['Longitude', 'Latitude']].values
nyc_service_requests[:2]

,Unique Key,Created Date,Agency Name,Complaint Type,Incident Zip,Incident Address,Latitude,Longitude
0,27325118,2014-02-03 23:59:26,New York City Police Department,Noise - Commercial,10128,163 EAST 92 STREET,40.783149,-73.952360
1,27325781,2014-02-03 23:58:29,Department of Parks and Recreation,Maintenance or Facility,11228,675 86TH STREET,40.619401,-74.022675


In [2]:
from geometryIO import get_transformPoint, proj4LL

# http://www.spatialreference.org/ref/epsg/3627/
proj4_nyc = (
    '+proj=lcc +lat_1=41.03333333333333 +lat_2=40.66666666666666 +lat_0=40.16666666666666 '
    '+lon_0=-74 +x_0=300000 +y_0=0 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs ')
transform_point = get_transformPoint(proj4LL, proj4_nyc)
untransform_point = get_transformPoint(proj4_nyc, proj4LL)
print tuple(nyc_service_request_locations[0])
print untransform_point(*transform_point(*nyc_service_request_locations[0]))

(-73.952359999999999, 40.783149000000002)
(-73.95236, 40.78314899999984)


In [3]:
import numpy as np

nyc_service_request_xys = np.array([
    transform_point(*_) for _ in nyc_service_request_locations])
nyc_service_request_xys[:2]

array([[ 304021.29010121,   68459.0557833 ],
       [ 298081.29727772,   50274.32558646]])

In [4]:
len(nyc_service_request_xys)

911

# Search local call history within radius

In [5]:
from geopy.geocoders import GoogleV3
geolocator = GoogleV3()
latitude, longitude = geolocator.geocode('345 Chambers St, New York, NY 10282')[1]
target_xy = transform_point(longitude, latitude)
target_xy

(298814.1634914433, 61220.092062934214)

In [6]:
radius_in_meters = 1600

from libraries.kdtree import KDTree
kdtree = KDTree(nyc_service_request_xys)
distances, indices = kdtree.query(target_xy, maximum_distance=radius_in_meters)
selected_rows = nyc_service_requests.ix[indices]
selected_rows

,Unique Key,Created Date,Agency Name,Complaint Type,Incident Zip,Incident Address,Latitude,Longitude
248,27325653,2014-02-03 16:55:33,Department of Consumer Affairs,Consumer Complaint,10013,310 GREENWICH STREET,40.717470,-74.010693
423,27325822,2014-02-03 14:06:39,Bureau of Fire Prevention - Alarm Unit - Inspe...,Fire Alarm - Reinspection,10007,101 BARCLAY STREET,40.713656,-74.011710
345,27317331,2014-02-03 15:27:20,Department of Health and Mental Hygiene,Indoor Air Quality,10007,14 MURRAY STREET,40.713287,-74.007750
421,27325638,2014-02-03 14:10:10,Bureau of Fire Prevention - Highrise Unit - In...,Fire Safety Director - F58,10038,109 PARK ROW,40.712584,-74.002490
429,27324508,2014-02-03 14:01:10,Bureau of Fire Prevention - Highrise Unit - In...,Fire Safety Director - F58,10038,109 PARK ROW,40.712584,-74.002490
684,27317287,2014-02-03 10:13:21,Operations Unit - Department of Homeless Services,Homeless Person Assistance,10038,110 WILLIAM STREET,40.708698,-74.007286
199,27323431,2014-02-03 17:54:32,Department of Consumer Affairs,Consumer Complaint,10013,43 MOTT STREET,40.715207,-73.998853
612,27320609,2014-02-03 11:18:48,Bureau of Fire Prevention - Highrise Unit - In...,Fire Safety Director - F58,10012,147 MERCER STREET,40.724878,-73.998517
616,27318313,2014-02-03 11:13:59,Bureau of Fire Prevention - Highrise Unit - In...,Fire Safety Director - F58,10012,147 MERCER STREET,40.724878,-73.998517
34,27318946,2014-02-03 22:17:36,Department of Consumer Affairs,Consumer Complaint,10002,87 BOWERY,40.717268,-73.995673


In [7]:
selected_rows['Complaint Type'].value_counts()

Fire Safety Director - F58    4
Consumer Complaint            3
Fire Alarm - Reinspection     1
Noise - Commercial            1
Homeless Person Assistance    1
Indoor Air Quality            1
dtype: int64